In [ ]:
import pandas 
import numpy as np 
import gc
import random
from collections import OrderedDict
from sklearn.preprocessing import * 
from keras.models import *
from keras.layers import *

In [ ]:

%pip install patool

from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/duolingo/processed.rar /content/

import patoolib
patoolib.extract_archive("processed.rar", outdir="/content")
!gzip -dv /content/processed.npy.gz

dataset = np.asarray(np.load('processed.npy')).astype(np.float)  
np.random.shuffle(dataset)  
dataset = dataset[0:100000]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 5.5 MB/s 
Mounted at /content/drive
patool: Extracting processed.rar ...
patool: running /usr/bin/unrar x -- /content/processed.rar
patool:     with cwd='/content'
patool: ... processed.rar extracted to `/content'.
gzip: /content/processed.npy.gz: No such file or directory


<ipython-input-2-b70a273dde7f>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset = np.asarray(np.load('processed.npy')).astype(np.float)


In [ ]:
# constants
MINIMUM_ROWS_PER_SEQUENCE = len(dataset[0])
MAX_AUGMENTATION_DELTA_0 = 1000

TRAIN_TEST_SPLIT = 0.9

In [ ]:
 
output = np.array(dataset[:,dataset.shape[1]-1, 0])
output = np.expand_dims(output, axis=1) 
  

In [ ]:
# creating the new dataset as:
# [previous_output, delta]
for row in range(dataset.shape[0]):
    for seq in range(MINIMUM_ROWS_PER_SEQUENCE-1, -1, -1):
        if seq == 0:
            dataset[row,seq,0] = -1
        else:
            dataset[row,seq,0] = dataset[row,seq-1,0] 
            if dataset[row,seq,0] == -1 and dataset[row,seq,1] != -1:
                dataset[row,seq,0] = 0
             

#normalization
 
# PERCHè non LAYER NORMALIZATION?
    
'''
normalized = []

index = 0
best = 0
for el in dataset:
  for seq in el:
    normalized.append(seq) 
    if (seq[1] / (3600*24)) > 60: 
        best = index
    index += 1 
normalized = np.array(normalized, dtype=np.float) 
# NOTA: con ReLU fai in modo di non avere ouput negativi
scaler = StandardScaler(with_mean = False)

scaler.fit(normalized[:,1:2])
normalized[:,1:2] /= 3600*24
#normalized[:,1:2] = scaler.transform(normalized[:,1:2]) 

#scaler.fit(normalized[:,4:5])
#normalized[:,4:5] = scaler.transform(normalized[:,4:5]) 

# fixing wrong normalization
for i in range(len(normalized)): 
    if normalized[i][0] == -1:
        for k in range(1, len(normalized[i])):
            normalized[i][k] = -1
 
'''
#dataset = np.array(np.array_split(normalized, len(normalized)/MINIMUM_ROWS_PER_SEQUENCE), dtype=np.float) 

#dataset = np.array()
# leave only p_recall, delta, session_seen
#dataset = dataset[:,:,[0,1,4]]

'\nnormalized = []\n\nindex = 0\nbest = 0\nfor el in dataset:\n  for seq in el:\n    normalized.append(seq) \n    if (seq[1] / (3600*24)) > 60: \n        best = index\n    index += 1 \nnormalized = np.array(normalized, dtype=np.float) \n# NOTA: con ReLU fai in modo di non avere ouput negativi\nscaler = StandardScaler(with_mean = False)\n\nscaler.fit(normalized[:,1:2])\nnormalized[:,1:2] /= 3600*24\n#normalized[:,1:2] = scaler.transform(normalized[:,1:2]) \n\n#scaler.fit(normalized[:,4:5])\n#normalized[:,4:5] = scaler.transform(normalized[:,4:5]) \n\n# fixing wrong normalization\nfor i in range(len(normalized)): \n    if normalized[i][0] == -1:\n        for k in range(1, len(normalized[i])):\n            normalized[i][k] = -1\n \n'

array([[1, 1],
       [2, 1],
       [1, 2]])

In [ ]:
# RNN input calc
rnn_input_test = dataset[int(len(dataset)*TRAIN_TEST_SPLIT):]
rnn_input = dataset[:int(len(dataset)*TRAIN_TEST_SPLIT)] 
output_test = output[int(len(output)*TRAIN_TEST_SPLIT):]
output = output[:int(len(output)*TRAIN_TEST_SPLIT)]
 

# momentaneamente usiamo solo train
# quando useremo test, dobbiamo fare finta di averlo solo alla fine
# ossia non lo normalizziamo all'inizio ma solo dopo l'allenamento
# dropout?

In [ ]:
# Here we do some data augmentation
# for each sample, we add another sample according to the last output

adding_input = []
adding_output = []
for i in range(int(len(rnn_input))):
  
    did = []
    if output[i] == 0:
        for i in range(8):
            last = np.array(rnn_input[i])
            last[-1][1] = random.randint(last[-1][1]+1, last[-1][1]+1+MAX_AUGMENTATION_DELTA_0)
            if last[-1][1] not in did:
                adding_output.append([0])
                adding_input.append(last)
                did.append(last[-1][1])
    elif rnn_input[i][-1][1] > 0: 
        for i in range(8):
            last = np.array(rnn_input[i])
            last[-1][1] = random.randint(0, last[-1][1]-1)
            if last[-1][1] not in did:
                adding_output.append([1])
                adding_input.append(last)
                did.append(last[-1][1])

print(len(rnn_input))
rnn_input = np.concatenate( (rnn_input,np.array(adding_input)) )
output = np.concatenate( (output,np.array(adding_output)) )
print(len(rnn_input))

 

90000
623651


In [ ]:
rnn_input[-3]


array([[ -1.,   1.],
       [  1.,   3.],
       [  0.,   1.],
       [  0.,   1.],
       [  1.,   3.],
       [  1.,   5.],
       [  0.,   1.],
       [  1.,   3.],
       [  1.,   5.],
       [  0.,   1.],
       [  1.,   3.],
       [  1.,   5.],
       [  0.,   1.],
       [  1.,   3.],
       [  1.,   5.],
       [  1.,   7.],
       [  1.,  15.],
       [  1.,  30.],
       [  0.,   1.],
       [  1., 224.]])

In [ ]:
# creating a weighted loss

'''
m = dict()
for i in rnn_input:
    if int(i[-1][1]) not in m:
        m[int(i[-1][1]) ] = 0
    m[int(i[-1][1])] += 1
'''

total_1 = sum([(1 if i[0] == 1 else 0) for i in output])
total_0 = sum([(1 if i[0] == 0 else 0) for i in output])
total_others = len(output)-total_1-total_0

# weights_calc = np.array([ 1.0/m[int(i[-1][1])] if m[int(i[-1][1])] > 1000 else 1.0/(m[1])  for i in rnn_input])
weights_calc = np.array([(1 if i[0] != 1 else 1) for i in output]) 

In [ ]:
output[12]

array([1.])

In [ ]:
# we do have 6 elements, each one with: p_recall, delta, history_seen, history_correct, session_seen, session_correct
# we keep 5 of those for the LSTM RNN
# we use another network with the 6th element and 5 features (all except p_recall)
# we connect the networks and try to predict the last p_recall

tf.keras.backend.clear_session() 
from keras import regularizers

rnn_layer_input = Input(shape=rnn_input[0].shape)
rnn_layer_0 = Masking(mask_value=-1.0)(rnn_layer_input)
rnn_layer_1 = GRU(20,dropout=0.1, recurrent_dropout=0.13, kernel_regularizer=regularizers.L2(0.001), recurrent_regularizer=regularizers.L2(0.001))(rnn_layer_0)
dense_layer_00 = Dropout(0.13)(Dense(20, activation='relu',  kernel_regularizer=regularizers.L2(0.001), activity_regularizer=regularizers.L2(0.01)) (rnn_layer_1))
dense_layer_down = Dense(2, activation="relu")(dense_layer_00)
dense_layer_000 = Dense(20, activation="sigmoid")(dense_layer_down)
dense_layer_0 = Dense(1, activation='sigmoid', name="cases") (dense_layer_000)  

model = Model([rnn_layer_input], [dense_layer_0])
metrics=["AUC", "MAE"]
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-2), loss="binary_crossentropy", metrics=metrics)


# beta regression 
#print(rnn_input[:6])
model.fit([rnn_input], output,sample_weight=weights_calc, epochs=10000, batch_size=10000, validation_data=([rnn_input_test], output_test))

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10000
63/63 [==============================] - 9s 99ms/step - loss: 0.4501 - auc: 0.8765 - MAE: 0.2892 - val_loss: 0.5490 - val_auc: 0.5925 - val_MAE: 0.3095
Epoch 2/10000
63/63 [==============================] - 8s 121ms/step - loss: 0.1801 - auc: 0.9682 - MAE: 0.1098 - val_loss: 0.5475 - val_auc: 0.6441 - val_MAE: 0.2641
Epoch 3/10000
63/63 [==============================] - 6s 90ms/step - loss: 0.1349 - auc: 0.9793 - MAE: 0.0724 - val_loss: 0.5417 - val_auc: 0.6460 - val_MAE: 0.2755
Epoch 4/10000
63/63 [==============================] - 6s 89ms/step - loss: 0.1228 - auc: 0.9831 - MAE: 0.0631 - val_loss: 0.5710 - val_auc: 0.6491 - val_MAE: 0.2557
Epoch 5/10000
63/63 [==============================] - 6s 89ms/step - loss: 0.1138 - auc: 0.9859 - MAE: 0.0591 - val_loss: 0.6046 - val_auc: 0.6496 - val_MAE: 0.2431
Epoch 6/10000
63/63 [==============================] - 6s 90ms/step - loss: 0.1110 - auc: 0.9866 - MAE: 0.0570 - val_loss: 0.5407 - val_auc: 0.6540 - val_MAE: 0.2701
Epo

KeyboardInterrupt: ignored

In [ ]:
print(rnn_input[7])
print(output[7])

[[-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00 -1.00000000e+

In [ ]:
model.evaluate([rnn_input_test[:,0,:], rnn_input_test[:,1,:],rnn_input_test[:,2,:],rnn_input_test[:,3,:],rnn_input_test[:,4,:],rnn_input_test[:,5,:], dense_input_test], output_test)

In [ ]:
rnn_input_test[3][-1][1]

3.249386574074074

In [ ]:
maxx = 0
for i in rnn_input:
    if i[-1][1] > maxx:
        maxx = i[-1][1]
print(maxx)

11.016168981481481


In [ ]:

  ind = np.random.randint(0, len(rnn_input_test))
  example_rnn = np.array(np.expand_dims(rnn_input_test[ind], 0))
  print(example_rnn) 
  print(ind)
 
  if True:
    x = []
    y = []
    for i in range(100):
      j = (i)
      x.append(j)
      
      for i in range(len(example_rnn[0])):
          example_rnn[0][i][0] = 1
          example_rnn[0][i][1] = 100
      
      example_rnn[0][-1][1] = j 
      
      #print(example_rnn)
      #print(model.predict([example_rnn],verbose = 0))
      y.append(model.predict([example_rnn],verbose = 0)[0][0])
!pip install matplotlib

import matplotlib.pyplot as plt 

# make data
x = np.array(x)
y = np.array(y)

# plot
fig, ax = plt.subplots()

ax.plot(x, y, linewidth=2.0)
 

plt.show()

 

[[[-1. -1.]
  [-1. -1.]
  [-1. -1.]
  [-1. -1.]
  [-1. -1.]
  [ 0.  0.]
  [ 0.  1.]
  [ 0.  1.]
  [ 1.  2.]
  [ 0.  1.]
  [ 1.  2.]
  [ 1.  5.]
  [ 1.  7.]
  [ 1. 12.]
  [ 0.  1.]
  [ 0.  1.]
  [ 1.  3.]
  [ 1.  5.]
  [ 1.  7.]
  [ 1. 15.]]]
7958
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
